In [ ]:
from datasets import load_dataset
dataset2 = load_dataset("Lancelot53/commonVoice_bn_train_preprocessed_trimmed")
dataset = load_dataset("Lancelot53/commonVoice_bn_validation_preprocessed")

In [ ]:
from datasets import concatenate_datasets
dataset =concatenate_datasets([dataset["validation"], dataset2["train"]])

In [ ]:
def trim_silence(batch):
    arr = batch['input_values']
    
    try:
        _max = max(max(arr), -min(arr))
        old_length = len(arr)
        
        threshold = 30

        for i,e in enumerate(arr):
            if threshold*e>_max:
                break

        for j,e in enumerate(reversed(arr)):
            if threshold*e>_max:
                break

        batch['input_values'] = arr[i:old_length-j]
        batch['input_length'] = old_length -i -j
    except:
        print(batch['input_length'])
    return batch

In [ ]:
dataset = dataset.map(trim_silence)

In [ ]:
max_input_length_in_sec = 10.0
dataset = dataset.filter(lambda x: x < max_input_length_in_sec * 16000, input_columns=["input_length"])
min_input_length_in_sec = 1
dataset = dataset.filter(lambda x: x > min_input_length_in_sec * 16000, input_columns=["input_length"])

In [ ]:
dataset = dataset.train_test_split(test_size=0.15, seed=4)

In [ ]:
dataset.save_to_disk("./45kData")